# Import and load test data

In [1]:
import os
os.chdir("/media/pc1/Ubuntu/Extend_Data/em_Thanh/medrag_colpali")
os.environ["HF_HOME"] = "/media/pc1/Ubuntu/Extend_Data/hf_models"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
import json
import pandas as pd
import fitz
import os
import re
import time

from src.rag.rag import Rag
from google.genai import types
from PIL import Image
from google import genai
from typing import List
from tqdm import tqdm
from dotenv import load_dotenv

os.chdir("/media/pc1/Ubuntu/Extend_Data/em_Thanh/medrag_colpali")
os.environ["HF_HOME"] = "/media/pc1/Ubuntu/Extend_Data/hf_models"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Configure the SDK
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY_2"))


/media/pc1/Ubuntu/Extend_Data/em_Thanh/medrag_colpali/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def extract_text_from_pdf(pdf_path):
        # Open the PDF
        doc = fitz.open(pdf_path)
        all_text = ""

        # Loop through each page and extract text
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)         
            text = page.get_text("text")           
            all_text += f"\n--- Page {page_num + 1} ---\n{text}"

        doc.close()
        return all_text

In [4]:
result_file = "data/processed/medgemma/results.json"
with open(result_file, "r", encoding="utf-8") as f:
    data = json.load(f)

In [5]:
data[0]

{'query': 'Intro:\n1 A 20-Year-Old Woman from Sudan With Fever, Haemorrhage and Shock DANIEL G. BAUSCH\n\nClinical Presentation:\nHistory A 20-year-old housewife presents to a hospital in northern Uganda with a 2-day history of fever, severe asthenia, chest and abdominal pain, nausea, vomiting, diarrhoea and slight non-productive cough. The patient is a Sudanese refugee living in a camp in the region. She denies any contact with sick people. Clinical Findings The patient is prostrate and semiconscious on admission. Vital signs: temperature 39.6°C, (103.3°F) blood pressure 90/60mmHg, pulse 90 bpm, and respiratory rate 24 cycles per minute. Physical examination revealed abdominal tenderness, especially in the right upper quadrant, hepatosplenomegaly and bleeding from the gums. The lungs were clear. No rash or lymphadenopathy was noted.\n\nQuestions:\n1. Is the patient’s history and clinical presentation consistent with a haemorrhagic fever (HF) syndrome? 2. What degree of nursing precaut

# Generate answer from only medgemma

In [4]:
def safe_append_json(filepath, record):
    # Ensure the file exists with a valid empty JSON array
    if not os.path.exists(filepath):
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump([], f)

    # Load safely, reset if corrupted
    with open(filepath, "r+", encoding="utf-8") as f:
        try:
            data = json.load(f)
            if not isinstance(data, list):
                data = []  # Force reset if JSON is not a list
        except json.JSONDecodeError:
            data = []  # Reset if file is empty or invalid

        data.append(record)

        # Overwrite with updated JSON
        f.seek(0)
        json.dump(data, f, indent=4, ensure_ascii=False)
        f.truncate()

In [3]:
path_file = "data/raw/cases_test"
files = os.listdir(path_file)
text_lst = []
img_lst = []

for file in files:
    file_path = os.path.join(path_file, file)
    imgs = []

    # Load JSON
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Combine all fields except "Extracted Figure"
    combined_text = ""
    for key, value in data.items():
        if key not in ["Extracted Figure", "Further Reading"]:
            combined_text += f"{key}:\n{value}\n\n"

    # Add Descriptions from "Extracted Figure"
    descriptions = []
    for figure_group in data.get("Extracted Figure", []):
        for figure in figure_group:
            desc = figure.get("Description", "")
            descriptions.append(desc)

    if descriptions:
        combined_text += "Figure Descriptions:\n"
        for desc in descriptions:
            combined_text += f"{desc}\n"

    # Get image links
    image_paths = []
    for figure_group in data.get("Extracted Figure", []):
        for figure in figure_group:
            path = figure.get("Link")
            path = os.path.join('data/raw', path)
            if path:
                image_paths.append(path)

    # Final combined text
    text_lst.append(combined_text)
    img_lst.append(image_paths)


In [5]:
output_file = "data/processed/medgemma/results.json"
processed_files = []
with open(output_file, "r", encoding="utf-8") as f:
    data = json.load(f)
for case in data:
    for key, value in case.items():
            if key in ['original_file']:
                processed_files.append(value)

print(len(processed_files))

93


In [10]:
for query, imgs, file in zip(text_lst, img_lst, files):
    if file not in processed_files:
        answer = rag.get_answer_from_medgemma(
            query=query, 
            images_path=imgs, 
        )
        print(answer)

        record = {
            "query": query,
            "images_used": imgs,
            "answer": answer,
            "original_file": file
        }

        # Load existing data, append, and rewrite
        safe_append_json(output_file, record)


Based on the provided information, the most likely diagnosis is **Crusted Scabies (Norwegian Scabies)**.

Here's why:

*   **Clinical Presentation:** The widespread, exfoliative rash, especially in the axillae, buttocks, and thighs, along with fissures on the wrists and knees, is highly characteristic of crusted scabies. The presence of flakes of skin on the mattress further supports this diagnosis.
*   **Patient Demographics:** The patient is an indigenous Australian woman living in a remote community, which is a known risk factor for endemic scabies.
*   **Systemic Symptoms:** The patient's fever, tachycardia, tachypnea, and low blood pressure indicate sepsis, which is a common complication of crusted scabies due to secondary bacterial infections.
*   **Laboratory Findings:** The skin scrapings confirmed the presence of scabies mites and eggs, supporting the diagnosis. The blood cultures revealed *Staphylococcus aureus* infection, which is a common secondary bacterial infection in cr

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Based on the provided information, the most likely diagnosis for the 7-year-old girl from Peru with the chronic skin ulcer on her nose is **cutaneous leishmaniasis**.

Here's the reasoning:

*   **Clinical Presentation:** The description of the lesion (localized ulcer with raised edges, cobblestone-patterned bottom) is highly characteristic of cutaneous leishmaniasis.
*   **Geographic Location:** The patient lives in Peru, a known endemic area for cutaneous leishmaniasis.
*   **Travel History:** The patient's recent travel to a valley on the western slopes of the Andes further supports this diagnosis, as these areas are known to be endemic for leishmaniasis.
*   **Positive Leishmanin Skin Test:** A positive leishmanin skin test is a strong indicator of past or current exposure to Leishmania parasites.
*   **PCR Confirmation:** The positive PCR result confirms the presence of Leishmania parasites.
*   **Species Identification:** Identification of *Leishmania (Viannia) peruviana* provide

# Context Relevance

In [4]:
def clean_nejm_text(text: str) -> str:
    """
    Cleans extracted text from NEJM PDFs by removing repeated watermarks,
    copyright notices, and headers/footers.

    Args:
        text (str): The raw extracted text from the PDF.

    Returns:
        str: The cleaned text without repeated boilerplate.
    """
    # Pattern to match NEJM boilerplate lines
    patterns = [
        r"The New England Journal of Medicine is produced by NEJM Group.*",
        r"Downloaded from nejm\.org by .* on .*\. For personal use only\.",
        r"No other uses without permission\. Copyright © \d{4} Massachusetts Medical Society\. All rights reserved\."
    ]
    
    cleaned_text = text
    for pattern in patterns:
        cleaned_text = re.sub(pattern, "", cleaned_text, flags=re.IGNORECASE)

    # Remove repeated "--- Page X ---" markers if present
    cleaned_text = re.sub(r"--- Page \d+ ---", "", cleaned_text)

    # Remove excessive blank lines (collapse multiple newlines to one)
    cleaned_text = re.sub(r"\n\s*\n+", "\n\n", cleaned_text)

    # Strip leading/trailing whitespace
    return cleaned_text.strip()

In [5]:
def extract_context(pdf_paths: List[str], result_kg: str) -> str:
    result = ""
    result = result + "## RETRIEVED CLINICAL CASES FROM VECTOR DATABASE\n"
    for i, pdf_path in enumerate(pdf_paths):
        result = result + f"The {i+1}th case:\n"
        result = result + clean_nejm_text(extract_text_from_pdf(pdf_path)) + "\n"

    result = result + f"## RETRIEVED DISEASES WITH THEIR SYMPTOMS FROM KNOWLEDGE GRAPH\n{result_kg}\n"

    return result


In [6]:
def load_images(images_used: List[str]):
    results = []
    for image in images_used:
        if image:
            with open(image, 'rb') as f:
                image_bytes = f.read()
            results.append(
                types.Part.from_bytes(
                    data=image_bytes,
                    mime_type='image/jpeg',
                ),
            )
        else:
            continue
    return results

In [7]:
def load_base_prompt_context_relevance(query: str, retrieved_content: str) -> str:
    return f"""
You are **Dr. Trop**, an infectious‑disease expert clinician and RAG auditor.

## Task
Judge whether the combined evidence (Part A + Part B) is sufficient
to arrive at the *correct diagnosis* for the clinical case in the query.
Ignore subsidiary questions in the case; focus on diagnosis only.

## Evidence Types
* **Part A – Similar Clinical Cases**  
  • Each snippet is a previously solved case that the vector DB says is
    most similar to the query case.  
* **Part B – Disease ⇄ Symptom Facts**  
  • Each snippet is a structured fact (disease, key symptoms, lab signs)
    from the knowledge graph, intended for differential diagnosis.

## Scoring Rubric
| Score | Meaning |
|-------|---------|
| **2** | Evidence *collectively* gives all facts needed for a confident, specific diagnosis. |
| **1** | Evidence is partially useful but omits critical findings or leaves >1 plausible diagnoses. |
| **0** | Evidence is irrelevant or clearly insufficient for diagnosis. |

## Evaluation Protocol
1. **Paraphrase** the query case in ≤1 sentence.  
2. **Analyse Part A (Similar Cases)** – for each snippet:  
   a. Summarise in one sentence.  
   b. Does it share decisive findings with the query? (Yes/No & why)  
   c. Assign a snippet score (0‑2).  
3. **Analyse Part B (Disease ⇄ Symptom Facts)** – for each snippet:  
   a. Summarise in one sentence.  
   b. State if it directly supports or refutes a likely diagnosis.  
   c. Assign a snippet score (0‑2).  
4. **Synthesis & Self‑Consistency** – reconcile insights from both parts; if reasoning paths diverge, rethink and choose the most consistent.  
5. **Overall Score** – output one final 0‑2 value using the rubric.  
6. If the evidence is clearly inadequate, say: *“I do not have sufficient information.”*  
7. **Important:** your very last character must be that single digit (0|1|2) with nothing after it.

## Few‑Shot Examples
### Example 1
*Query:* “30‑year‑old in Vietnam with paroxysmal fever, splenomegaly, and ring‑form parasites on blood smear.”  
*Part A Snippet:* “Case of falciparum malaria treated successfully with DHA‑PPQ.”  
*Part B Snippet:* “Falciparum malaria → fever, anemia, splenomegaly; thick smear positive for ring forms.”  
*Evaluation:*  
- Part A snippet: score 2 (high match)  
- Part B snippet: score 2 (directly supports)  
**Overall = 2**

### Example 2
*Query:* “Child with rash and cough; query seeks measles vs rubella.”  
*Part A Snippet:* “Rash illness caused by parvovirus B19.”  
*Part B Snippet:* “Measles: cough, coryza, conjunctivitis, Koplik spots.”  
*Evaluation:*  
- Part A snippet: score 0 (different disease)  
- Part B snippet: score 1 (partial—gives measles signs but no rash description)  
**Overall = 1**

---

# QUERY
{query}

# RETRIEVED CONTEXT  
{retrieved_content}

# YOUR ANALYSIS AND FINAL SCORE"""


In [8]:
def safe_append_json(filepath, record):
    # Ensure the file exists with a valid empty JSON array
    if not os.path.exists(filepath):
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump([], f)

    # Load safely, reset if corrupted
    with open(filepath, "r+", encoding="utf-8") as f:
        try:
            data = json.load(f)
            if not isinstance(data, list):
                data = []  # Force reset if JSON is not a list
        except json.JSONDecodeError:
            data = []  # Reset if file is empty or invalid

        data.append(record)

        # Overwrite with updated JSON
        f.seek(0)
        json.dump(data, f, indent=4, ensure_ascii=False)
        f.truncate()

In [9]:
evaluated_data_context_relevance = []

In [11]:
evaluated_file = "data/processed/results_context_relevance.json"

processed_files = []
with open(evaluated_file, "r", encoding="utf-8") as f:
    processed_data = json.load(f)
for case in processed_data:
    for key, value in case.items():
            if key in ['original_file']:
                processed_files.append(value)

for test in tqdm(data):
    if test["original_file"] in processed_files:
        continue

    query_text = load_base_prompt_context_relevance(
        test["query"],
        extract_context(test["pdf_paths"], test["results_kg"])
    )

    image_parts = load_images(test["images_used"])  

    contents = []
    if image_parts:                                 
        contents.append(image_parts)                 
    contents.append(query_text)                      

    response = client.models.generate_content(
        model="gemini-2.0-flash-lite",
        contents=contents
    )

    test["evaluation"] = response.text
    test["score"] = response.text[-2]
    evaluated_data_context_relevance.append(test)
    safe_append_json(evaluated_file, test)
    time.sleep(4)

100%|██████████| 93/93 [16:24<00:00, 10.59s/it]


# Answer Faithfulness

In [ ]:
def load_base_prompt_answer_faithfulness(
    query: str,
    draft_answer: str,
    retrieved_content: str,
) -> str:

    return f"""
You are **Dr. Trop**, an infectious‑disease specialist and RAG auditor.

## Task
Evaluate whether the *Draft Answer* below is **faithful** to the combined evidence
(Part A + Part B).  Judge factual support—NOT style or completeness.

## Evidence Types
* **Part A – Similar Clinical Cases**  
* **Part B – Disease ⇄ Symptom Facts**

## Faithfulness Scoring Rubric
| Score | Meaning |
|-------|---------|
| **2** | Every key claim in the draft answer is directly supported by ≥1 evidence snippet, and no claim contradicts the evidence. |
| **1** | Most claims are supported, but ≥1 minor claim lacks support **or** minor inconsistencies exist. |
| **0** | Major claims are unsupported or contradicted; hallucination or clear conflict with evidence. |

## Evaluation Protocol
1. **Restate** the draft answer’s main diagnosis in ≤1 sentence.  
2. **Claim‑by‑claim check**: For each distinct factual claim in the draft answer:  
   a. Quote or paraphrase the claim (≤15 words).  
   b. Cite supporting snippet IDs from Parts A/B *or* mark “No Support” / “Contradicted”.  
3. **Conflict scan**: Note any claim that opposes evidence.  
4. **Faithfulness Verdict**: Decide overall score (0‑2) using the rubric.  
5. If >50 % of claims lack support, explicitly say: *“Answer largely unsupported.”*  
6. **Important:** Finish your entire reply with the single digit score (0|1|2) and nothing after it.

## Few‑Shot Examples
### Example 1  (Score 2)
*Draft Answer:* “The patient has falciparum malaria; ACT (DHA‑PPQ) is indicated.”  
*Evidence snippets:* (Part A case of falciparum, Part B symptom profile).  
*Evaluation:* All claims supported → Overall = 2

### Example 2  (Score 0)
*Draft Answer:* “Child most likely has parvovirus B19 rash illness.”  
*Evidence snippets:* Part A & B highlight measles signs (Koplik spots) only.  
*Evaluation:* Central claim contradicted → Overall = 0

---

# QUERY (clinical case)
{query}

# DRAFT ANSWER TO EVALUATE
{draft_answer}

# RETRIEVED CONTEXT  
{retrieved_content}

# YOUR ANALYSIS AND FINAL SCORE"""


In [13]:
evaluated_data_answer_faithfulness = []

In [19]:
evaluated_file = "data/processed/results_answer_faithfulness.json"

processed_files = []
with open(evaluated_file, "r", encoding="utf-8") as f:
    processed_data = json.load(f)
for case in processed_data:
    for key, value in case.items():
            if key in ['original_file']:
                processed_files.append(value)

for test in tqdm(data):
    if test["original_file"] not in processed_files:
        query_text = load_base_prompt_answer_faithfulness(
            query=test["query"],
            draft_answer=test["answer"],
            retrieved_content=extract_context(test["pdf_paths"], test["results_kg"])
        )

        image_parts = load_images(test["images_used"])  

        contents = []
        if image_parts:                                 
            contents.append(image_parts)                 
        contents.append(query_text)                      

        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",
            contents=contents
        )

        test["evaluation"] = response.text
        test["score"] = response.text[-1]
        evaluated_data_answer_faithfulness.append(test)
        safe_append_json(evaluated_file, test)
        time.sleep(4)

100%|██████████| 93/93 [11:50<00:00,  7.64s/it]


# Answer Relevance

In [22]:
def load_base_prompt_answer_relevance(
    query: str,
    draft_answer: str,
    retrieved_content: str,
) -> str:
    return f"""
You are **Dr. Trop**, an infectious‑disease specialist and Retrieval‑Augmented Generation auditor.

## Task
Assess the **Answer Relevance**: Does the *Draft Answer* appropriately address
the clinical‑case query **and** draw on the information contained in the
retrieved context (Part A + Part B)?

## Evidence Types
* **Part A – Similar Clinical Cases**  
* **Part B – Disease ⇄ Symptom Facts**

## Relevance Scoring Rubric
| Score | Meaning |
|-------|---------|
| **2** | Answer directly and completely addresses the query’s diagnostic goal, uses facts consistent with ≥1 retrieved snippet, and avoids off‑topic content. |
| **1** | Answer touches on the query but is incomplete **or** includes minor off‑topic / unsupported material. |
| **0** | Answer does not meaningfully address the query, or is dominated by content unrelated to the retrieved context. |

## Evaluation Protocol
1. **Restate** the essential question being asked (≤1 sentence).  
2. **Summarise** the draft answer’s key points (≤2 sentences).  
3. **Relevance Check**:  
   a. Does the answer directly solve the diagnostic question? (Yes/No & why)  
   b. Identify which snippets (Part A/B) the answer appears to rely on—or state “No linkage”.  
   c. Note any off‑topic or speculative content.  
4. **Verdict**: Choose a final relevance score (0‑2) using the rubric.  
5. If the answer barely references the retrieved evidence, say: *“Answer minimally grounded.”*  
6. **Important:** End your whole reply with that single digit (0|1|2) and nothing after it.

## Few‑Shot Examples
### Example 1 (Score 2 – Highly Relevant)
*Query:* “30‑year‑old with paroxysmal fever and ring‑form parasites—diagnosis?”  
*Draft Answer:* “These findings indicate falciparum malaria; ACT such as DHA‑PPQ is recommended.”  
*Context:* Part A similar malaria case; Part B malaria symptom profile.  
*Why:* Directly answers; cites same diagnostic facts → **Overall = 2**

### Example 2 (Score 1 – Partially Relevant)
*Query:* “Child with fever and purpuric rash—differential?”  
*Draft Answer:* “Could be meningococcemia. Start ceftriaxone.”  
*Context:* Part A lists meningococcemia & dengue; Part B symptom facts.  
*Why:* Mentions one plausible diagnosis but ignores dengue; no link to platelet findings → **Overall = 1**

### Example 3 (Score 0 – Irrelevant)
*Query:* “Identify viral family of dengue virus.”  
*Draft Answer:* “Treat dengue shock with IV fluids.”  
*Context:* Part A taxonomic case; Part B Flaviviridae facts.  
*Why:* Talks about treatment, not taxonomy → **Overall = 0**

---

# QUERY (clinical case)
{query}

# DRAFT ANSWER TO EVALUATE
{draft_answer}

# RETRIEVED CONTEXT  
{retrieved_content}

# YOUR ANALYSIS AND FINAL SCORE"""

In [23]:
evaluated_data_answer_relevance = []

In [45]:
evaluated_file = "data/processed/results_answer_relevance.json"

processed_files = []
with open(evaluated_file, "r", encoding="utf-8") as f:
    processed_data = json.load(f)
for case in processed_data:
    for key, value in case.items():
            if key in ['original_file']:
                processed_files.append(value)

for test in tqdm(data):
    if test["original_file"] not in processed_files:
        query_text = load_base_prompt_answer_relevance(
            query=test["query"],
            draft_answer=test["answer"],
            retrieved_content=extract_context(test["pdf_paths"], test["results_kg"])
        )

        image_parts = load_images(test["images_used"])  

        contents = []
        if image_parts:                                 
            contents.append(image_parts)                 
        contents.append(query_text)                      

        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",
            contents=contents
        )

        test["evaluation"] = response.text
        test["score"] = response.text[-1]
        evaluated_data_answer_relevance.append(test)
        safe_append_json(evaluated_file, test)
        time.sleep(4)

100%|██████████| 93/93 [00:15<00:00,  6.16it/s]


# Result

In [48]:
files = ["data/processed/results_context_relevance.json", "data/processed/results_answer_faithfulness.json", "data/processed/results_answer_relevance.json"]

In [27]:
def score_str_to_int(s: str, valid: set[str] = {"0", "1", "2"}) -> int:
    if not s:
        return None
    last = s.strip()[-1]
    if last in valid:
        return int(last)
    return None

In [49]:
for file in files:
    res = 0
    print(f"Evaluation of {file[23:-5]}")
    with open(file, "r", encoding="utf-8") as f:
        eval_data = json.load(f)
    for i, case in enumerate(eval_data):
        # print(f"{i} {case['score']}\n{case['original_file']}")
        res += score_str_to_int(case["score"])
    res /= (len(eval_data) * 2)
    print(res)


Evaluation of context_relevance
0.3655913978494624
Evaluation of answer_faithfulness
0.3010752688172043
Evaluation of answer_relevance
0.20430107526881722
